In [ ]:
from transformers import AutoTokenizer, TextDataset,DataCollatorForLanguageModeling, Trainer, TrainingArguments,AutoModelWithLMHead
import torch
from transformers import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split

def preprocessing(path):
	with open(path, 'r') as f:
		text = f.read().replace('"',"").replace("«","").replace("«","»").replace("‘","").split('\n\n')
	text = list(map(lambda x: "<BOS>"+x.replace("\n"," "), text))
	return text

tokenizer = AutoTokenizer.from_pretrained("GroNLP/gpt2-small-italian-embeddings", max_lenght=256)
model = AutoModelWithLMHead.from_pretrained("GroNLP/gpt2-small-italian-embeddings")

input_data = "divina_commedia.txt"

text = preprocessing(input_data)

device = torch.device("cuda")

model = model.to(device)

train, eval = train_test_split(text, train_size=.9, random_state=2020)

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<EOS>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<EOS>".join(eval))

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset('train_tmp.txt','eval_tmp.txt',tokenizer)

training_args = TrainingArguments(
    output_dir="./dantebert", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=1500, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()
trainer.save_model()

In [ ]:
from transformers import pipeline

dante = pipeline('text-generation',model='./dantebert', tokenizer='GroNLP/gpt2-small-italian-embeddings')
dante("E colei che amai ")[0]['generated_text']

loading configuration file ./dantebert/config.json
Model config GPT2Config {
  "_name_or_path": "GroNLP/gpt2-small-italian-embeddings",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 0,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 100,
      "no_repeat_ngram_size": 4,
      "num_beams": 10,
      "repetition_penalty": 10.0,
      "temperature": 2.0,
      "top_k": 20,
 

'E colei che amai uopo più si vuoli, d’altrui lume già bianche e fioche, quasi come piante novelle rinognate per l’acqua fosser pronte.e disse: Piglia quel seme a li occhi; volgi ’l viso, e fammi nota la larghezza di questa nutrice».S’el s’aunasse ancor tutta la gente che già, in su la fortunata terra di Puglia, fu del suo sangue'